# 測試Job資料清洗與套用模型

## 1. Read Jobs from JSON

In [54]:
import json
import pymongo

In [55]:
def open_json_file(CACHE_FNAME):
    try:
        cache_file = open(CACHE_FNAME, 'r', encoding='utf-8-sig')
        cache_contents = cache_file.read()
        CACHE_DICTION = json.loads(cache_contents, encoding='utf-8-sig')
        cache_file.close()
        return CACHE_DICTION

    except:
        CACHE_DICTION = {}
        return CACHE_DICTION

In [56]:
def mongo_connect_build(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") 

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    mycol = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立

def data_find():
    # 尋找多筆資料
    return mycol.find({},{'_id':0})

In [57]:
mongo_connect_build('jieba_clean','job_clean')
job_data = data_find()

In [58]:
job_clean = []
for i in job_data:
    job_dict = {}
    job_dict['jobURL'] = i['jobURL']
    job_dict['jobDescription_concate_jieba'] = i['jobDescription_concate_jieba'][0]
    job_clean.append(job_dict)

In [59]:
job_clean[0].keys()

dict_keys(['jobURL', 'jobDescription_concate_jieba'])

In [60]:
job_clean[0]

{'jobURL': '4g03i',
 'jobDescription_concate_jieba': ['介紹',
  '銷售',
  '商品',
  '顧客',
  '接待',
  '服務',
  '電話',
  '諮詢',
  '調貨',
  '包裝',
  '退換貨',
  '商品',
  '進貨',
  '入庫',
  '銷售',
  '管理',
  '庫存',
  '管理',
  '商品',
  '包裝',
  '陳列',
  '促銷品',
  '換檔',
  '維持',
  '店櫃',
  '週遭',
  '整潔',
  '提升',
  '業績',
  '淑女',
  '服飾',
  '銷售',
  '服務',
  '門市',
  '店員',
  '專櫃',
  'Windows',
  'Outlook']}

In [53]:
cv_clean[0].keys()

dict_keys(['id', 'value'])

In [77]:
job_clean[2]['jobDescription_concate_jieba']

['成本會計',
 '三角貿易',
 '會計',
 '進貨',
 '銷貨',
 '費用',
 '單據',
 '審核',
 '傳票',
 '異動',
 '撥補',
 '帳務',
 '會計',
 '報表',
 '編製',
 '操作',
 '系統',
 '採購',
 '庫存',
 '管理',
 '進銷存',
 '管理',
 '廠商',
 '接洽',
 '訂單',
 '會計',
 '軟體',
 '財務',
 '報表',
 '會計',
 '軟體',
 '記帳',
 '出納',
 '一般會計',
 '會計學',
 '一般商業',
 '英文',
 'Excel',
 'Outlook',
 'PowerPoint',
 'Word',
 'Adobe',
 'Photoshop']

## 2. data processing and model prediction 

[Python开发 之 Sklearn的模型 和 CountVectorizer 、Transformer 保存 和 使用](https://blog.csdn.net/u014597198/article/details/103037709)

In [78]:
test_job = [' '.join(job_clean[2]['jobDescription_concate_jieba'])] #取一筆CV測試

In [79]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing
from joblib import dump, load
import pickle

In [80]:
def job_category_predict(jobdata):

    vectorizer = pickle.load(open("./vectorizer.pickel", "rb")) 
    transform_content = vectorizer.transform(jobdata) # CountVectorizer transform
    X_test = transform_content.toarray()
    # vectorizer.inverse_transform(X_test)

    model = load('./MultinomialNB.joblib') 
    y_pred = model.predict(X_test)  # MultinomialNB transform

    le = pickle.load(open("./le.pickel", "rb"))
    y_pred_category = le.inverse_transform(y_pred)
    y_pred_category = y_pred_category[0]
    
    return y_pred_category


In [81]:
test_job_category = job_category_predict(test_cv)

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [82]:
print(test_job) # raw data
print('='*20)
print(test_job_category)

['成本會計 三角貿易 會計 進貨 銷貨 費用 單據 審核 傳票 異動 撥補 帳務 會計 報表 編製 操作 系統 採購 庫存 管理 進銷存 管理 廠商 接洽 訂單 會計 軟體 財務 報表 會計 軟體 記帳 出納 一般會計 會計學 一般商業 英文 Excel Outlook PowerPoint Word Adobe Photoshop']
財會╱金融專業類


## 3. 連線至履歷，利用條件篩選

In [83]:
import pymongo

In [84]:
def mongo_connect_build(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") 

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    mycol = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立


In [85]:
mongo_connect_build('cakeresume','NoEng_jieba_resume')
cv_data = data_find()

In [86]:
job_data = [i for i in data_find()]

print(len(job_data))

610


In [87]:
for i in job_data[0:2]:
    print(i['id'])
    print(i['value'][0])
    print('='*20)

yo
['David', 'Huang', 'github', '很聰明', '助人', '解決', '疑難雜症', '工程', 'Education', '交通', '大學', '資訊', '管理', '財務', '金融', '資訊', '管理', '交通', '大學', '資訊', '工程', '輔系', '交通', '大學', '電機', '工程', '研究所', '計算機', '工程', 'Lab', 'Intelligent', 'Communication', 'Computing', 'CC', 'Experience', '跨出', '挑戰', '工研院', '機械', '實習生', '研究', '半導體', '微波', '退火', '技術', '低頻', '微波', '製程', '提升', '晶圓', '勻性', '協助', '實驗', '元件', '組裝', '橘子', '程式', '設計', '學苑', '助教', '協助', 'App', 'inventor', '營隊', '課程', 'Javascript', '課程', '協助', '講師', '教學', '技巧', '騏驥坊', '教育', '講師', 'IOT', '物聯網', '智慧', '營隊', '課程', '協助', '講師', '協調', '業務', '系統性', '創新', '網路', '工程', '軟硬體', '設備', '維護', '網站', '後端', 'PHP', 'Mysql', '前端', 'HTML', 'CSS', 'Javascript', '歐兔', '全球', '前後端', '網頁', '工程', '網頁', '前後端', '撰寫', '功能', '設計', '排版', '設計', '資料庫', '維護', '新創', '強短', '客戶', '障礙', 'PHP', 'Mysql', '前端', '語言', 'Codeigniter', '框架', '核果', '資訊', '學苑', '講師', '教材', '設計', '講課', '教授', 'Python', '課程', '學生', '程式', '教育', '社會', '挑戰', 'Python', '教材', '程式', '開發', '教材', '層級', '流程', '控制', '資料', 

## 4. 將結巴後的斷詞，利用CountVectorizer轉換減少字詞數量

In [88]:

 # 將結巴後的斷詞，利用CountVectorizer轉換減少字詞數量
def turn_content_BOW(content):
    X_content = [ " ".join(content)]
    vectorizer = pickle.load(open("./vectorizer.pickel", "rb")) 
    transform_content = vectorizer.transform(X_content) 
    X_content_array = transform_content.toarray()
    return X_content_array


# transform a tokenized job to BOW
def turn_job_BOW(jobdata):
    job_BOW_dict = {}
    
    # 將結巴後的斷詞，利用CountVectorizer轉換減少字詞數量
    job_content = turn_content_BOW(jobdata['value'][0])
    
    job_BOW_dict['id'] = jobdata['id']
    job_BOW_dict['value'] = job_content
    return job_BOW_dict

## 5. 計算cosine_similarity

[Overview of Text Similarity Metrics in Python](https://towardsdatascience.com/overview-of-text-similarity-metrics-3397c4601f50)

[Python: tf-idf-cosine: to find document similarity](https://stackoverflow.com/questions/12118720/python-tf-idf-cosine-to-find-document-similarity)

In [89]:
from sklearn.metrics.pairwise import cosine_similarity

In [90]:
def compute_similarity(jobdata, lst_cv):
    dict_prob = {}
    
    # transform a tokenized CV to BOW
    cv_BOW = turn_content_BOW(cv)
    
    for i in lst_jobdata:
        # transform a tokenized job to BOW
        job_BOW_dict = turn_job_BOW(i)
        
        job_id = job_BOW_dict['id']
        job_BOW = job_BOW_dict['value']
 
        job_prob = cosine_similarity(cv_BOW, job_BOW)       
        dict_prob[job_id] = job_prob[0][0]
        
    
    dict_prob = sorted(dict_prob.items(), key=lambda x: x[1], reverse = True)
    return dict_prob


In [91]:
result = compute_similarity(test_job, cv_data)

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

...

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\sklearn\base.py:334: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.1 w

In [92]:
# https://www.104.com.tw/job/5tz69

result[0:10]

[('linfe', 0.20184040364963407),
 ('nicole-tu', 0.19646908900284496),
 ('fendy326', 0.19552719646566083),
 ('shen-po-chun', 0.19276448081894737),
 ('fmvy5513', 0.19030929483520365),
 ('johnson', 0.18306640301880361),
 ('24a556', 0.18112269284728896),
 ('lee9505', 0.17745398113214778),
 ('xanxus', 0.16975262433626354),
 ('loveskymore', 0.16940786960184825)]